In [52]:
import numpy as np
import pandas as pd
from sklearn.model_selection import ShuffleSplit
import xgboost as xgb
import auxMethods as auxMethods

# Import the training file.



In [53]:
train_file = "train_data.csv"
df = pd.read_csv(train_file)
df

,Unnamed: 0,Age,Gender,Stage,GeneticRisk,TreatmentType,ComorbidityIndex,TreatmentResponse,SurvivalTime,Censored
0,0,76.0,1,4,NaN,0,1.0,0.0,2.9,0
1,1,63.0,1,3,0.0,0,1.0,0.0,4.7,0
2,2,64.0,0,2,0.0,0,0.0,0.0,NaN,0
3,3,56.0,0,3,1.0,0,1.0,0.0,2.9,0
4,4,67.0,1,3,1.0,0,3.0,1.0,3.0,0
...,...,...,...,...,...,...,...,...,...,...
395,395,50.0,1,2,0.0,0,NaN,0.0,NaN,0
396,396,62.0,1,1,0.0,0,0.0,1.0,NaN,0
397,397,59.0,0,2,1.0,0,NaN,1.0,4.3,0
398,398,65.0,1,3,0.0,0,1.0,1.0,2.5,0


# Check which values are missing

In [54]:
#print(df.isnull())
print(df.isnull().sum())

Unnamed: 0            0
Age                   0
Gender                0
Stage                 0
GeneticRisk          80
TreatmentType         0
ComorbidityIndex     70
TreatmentResponse    21
SurvivalTime         90
Censored              0
dtype: int64


# Fill the missing values with the mean value of the column


In [55]:
mean_GeneticRisk = df['GeneticRisk'].mean()
print(mean_GeneticRisk)
df['GeneticRisk'].fillna(mean_GeneticRisk,inplace=True)


0.275


In [56]:
# Value is an integer so the mean doesn't work.
mode_ComorbilityIndex = df['ComorbidityIndex'].mode()[0]
print(mode_ComorbilityIndex)
df['ComorbidityIndex'].fillna(mode_ComorbilityIndex,inplace=True)


1.0


In [57]:
# We try with the mode and with a missing string, since it depends on each persons genetics
mode_TreatmentResponse = df['TreatmentResponse'].mode()[0]
print(mode_TreatmentResponse)
df['TreatmentResponse'].fillna(mode_TreatmentResponse,inplace=True)

1.0


In [58]:
#Check the mean of survival time when the value of the censored column is 0
mean_SurvivalTime =  df[df['Censored']==0]['SurvivalTime'].mean()
print(mean_SurvivalTime)
df['SurvivalTime'].fillna(mean_SurvivalTime,inplace=True)

3.771563981042654


In [59]:
df

,Unnamed: 0,Age,Gender,Stage,GeneticRisk,TreatmentType,ComorbidityIndex,TreatmentResponse,SurvivalTime,Censored
0,0,76.0,1,4,0.275,0,1.0,0.0,2.900000,0
1,1,63.0,1,3,0.000,0,1.0,0.0,4.700000,0
2,2,64.0,0,2,0.000,0,0.0,0.0,3.771564,0
3,3,56.0,0,3,1.000,0,1.0,0.0,2.900000,0
4,4,67.0,1,3,1.000,0,3.0,1.0,3.000000,0
...,...,...,...,...,...,...,...,...,...,...
395,395,50.0,1,2,0.000,0,1.0,0.0,3.771564,0
396,396,62.0,1,1,0.000,0,0.0,1.0,3.771564,0
397,397,59.0,0,2,1.000,0,1.0,1.0,4.300000,0
398,398,65.0,1,3,0.000,0,1.0,1.0,2.500000,0


In [60]:
df.isnull().sum()

Unnamed: 0           0
Age                  0
Gender               0
Stage                0
GeneticRisk          0
TreatmentType        0
ComorbidityIndex     0
TreatmentResponse    0
SurvivalTime         0
Censored             0
dtype: int64

In [61]:
df.head

<bound method NDFrame.head of      Unnamed: 0   Age  Gender  Stage  GeneticRisk  TreatmentType  \
0             0  76.0       1      4        0.275              0   
1             1  63.0       1      3        0.000              0   
2             2  64.0       0      2        0.000              0   
3             3  56.0       0      3        1.000              0   
4             4  67.0       1      3        1.000              0   
..          ...   ...     ...    ...          ...            ...   
395         395  50.0       1      2        0.000              0   
396         396  62.0       1      1        0.000              0   
397         397  59.0       0      2        1.000              0   
398         398  65.0       1      3        0.000              0   
399         399  59.0       1      1        0.000              0   

     ComorbidityIndex  TreatmentResponse  SurvivalTime  Censored  
0                 1.0                0.0      2.900000         0  
1                 1